In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from scrapy import Selector
import pandas as pd
import yfinance as yf
import time

# Helper Functions
def scrape_nse_data():
    url = "https://www.nseindia.com/live_market/dynaContent/live_watch/equities_stock_watch.htm"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    }

    # Using requests to scrape NSE data
    session = requests.Session()
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract stock details (PE, EPS, 52-week high/low, etc.)
    stock_data = []
    rows = soup.find_all('tr')
    for row in rows[1:]:  # Skip header
        cols = row.find_all('td')
        if len(cols) > 0:
            stock = {
                "Stock": cols[0].text.strip(),
                "LTP": cols[1].text.strip(),
                "Change %": cols[2].text.strip(),
                "52 Week High": cols[3].text.strip(),
                "52 Week Low": cols[4].text.strip(),
                "Market Cap": cols[5].text.strip(),
            }
            stock_data.append(stock)
    return pd.DataFrame(stock_data)

def scrape_screener_data():
    url = "https://www.screener.in/company/"
    driver_path = "<path_to_chromedriver>"
    options = Options()
    options.add_argument("--headless")

    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    time.sleep(5)
    page_source = driver.page_source
    driver.quit()

    selector = Selector(text=page_source)

    # Extract data from Screener using Scrapy
    stock_data = []
    rows = selector.xpath('//table[@class="data-table"]//tr')
    for row in rows:
        stock = {
            "Stock": row.xpath('./td[1]/text()').get(),
            "PE": row.xpath('./td[2]/text()').get(),
            "EPS": row.xpath('./td[3]/text()').get(),
        }
        stock_data.append(stock)

    return pd.DataFrame(stock_data)

def get_stock_returns(stock_list):
    data = []
    for stock in stock_list:
        try:
            ticker = yf.Ticker(stock)
            hist = ticker.history(period="5y")

            # Calculate returns
            close = hist['Close']
            six_month_return = ((close[-1] / close[-126]) - 1) * 100 if len(close) >= 126 else None
            one_year_return = ((close[-1] / close[-252]) - 1) * 100 if len(close) >= 252 else None
            five_year_return = ((close[-1] / close[0]) - 1) * 100 if len(close) >= 1260 else None

            data.append({
                "Stock": stock,
                "6 Month Return": six_month_return,
                "1 Year Return": one_year_return,
                "5 Year Return": five_year_return,
            })
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")

    return pd.DataFrame(data)

# Main Execution
if __name__ == "__main__":
    nse_data = scrape_nse_data()
    screener_data = scrape_screener_data()

    # Merge DataFrames
    stock_data = pd.merge(nse_data, screener_data, on="Stock", how="outer")

    # Get stock returns
    stock_returns = get_stock_returns(stock_data['Stock'].tolist())

    # Final DataFrame
    final_df = pd.merge(stock_data, stock_returns, on="Stock", how="outer")
    print(final_df.head())

    # Save to CSV
    final_df.to_csv("nifty50_stock_data.csv", index=False)